## Business Understanding

I am to examine the data, as a Data Scioentist. the King County House Sales dataset to be specific, which can be found in kc_house_data.csv.I will be attempting to find the major contributars to house pricing  in a northwestern county.

I will be using several models of linear regression and multiple linear regresions.

In [11]:
# Importing the relevant libraries.

import pandas as pd

import statsmodels.api as sm

import matplotlib.pyplot as plt

%matplotlib inline 

import seaborn as sns


In [13]:
kc = pd.read_csv('kc_house_data.csv')
kc.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,...,7 Average,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7 Average,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,6 Low Average,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,7 Average,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,8 Good,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


### Dataframe exploration

In [19]:
#let us define some functions to get relevant information from the Data Frames.

#get value types of entries per column.

def get_value(data,col_name):
    
    return data[col_name].value_counts()

#get general information for the whole data frame.

def get_info(data):
    info_ = data.info()
    return info_


# Defining a function to get the percentages of missing values.
def missing_values(data):
    miss_vals = data.isnull().sum().sort_values(ascending=False)
    
    #percentages
    percentages = (((data.isnull().sum()) / len(data)).sort_values(ascending=False))*100
    
    #create dataframe of missing values
    
    missing_df = pd.DataFrame({"Total missing values": miss_vals, 'Percentage(%)':percentages})
    
    #if percentage == 0 implies no missing values
    missing_df.drop(missing_df[missing_df['Percentage(%)']==0].index, inplace = True)
    
    return missing_df

# defining a function to look for duplicates from the DataFrames.

def get_duplicate(data):
    duplicate = data.duplicated().value_counts()
    return duplicate

#define a function that returns the unique values in a column.

def get_unique(data,column):
    return data[column].unique()


#define a function that returns a list of column names with potential descrete data based of data type

def descrete(data):
    descrete_var_col_names = []
    categoricals = kc.select_dtypes("object")

    for col in categoricals:
        descrete_var_col_names.append(col)
    return descrete_var_col_names



In [9]:
# getting a summary or general info of the data frame

get_info(kc)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  object 
 9   view           21534 non-null  object 
 10  condition      21597 non-null  object 
 11  grade          21597 non-null  object 
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

In [8]:
#looking for any missing values
missing_values(kc)


,Total missing values,Percentage(%)
yr_renovated,3842,17.789508
waterfront,2376,11.001528
view,63,0.291707


### 3 columns observed with missing data 


In [10]:
# checking for duplicates on the dataframe.
get_duplicate(kc)

False    21597
dtype: int64

In [17]:
# potential descrete variables.
descrete(kc)

['date', 'waterfront', 'view', 'condition', 'grade', 'sqft_basement']

# Data Understanding of the King County House Sales dataset.

It has been loaded in with pandas as kc. Short for King County.

It has 21597 rows and 21 columns.

No duplicate rows were observed.

Only 3 columns were observed with missing values;

> yr_renovated  17.7% missing

> waterfront    11.0% missing

> view          0.29% missing


# Data Preparation of the King County House Sales dataset.



In [22]:
# dealing with the missing values.

# since the percent of missing values is low and i may use there column
# i will drop the rows with the missing values.

kc.dropna(inplace = True)

# rechecking missing values

missing_values(kc)

,Total missing values,Percentage(%)


In [24]:
kc.shape

(15762, 21)

The shape has changed from (21597,21) , to (15762, 21) after dropping the values... if I wount use the columns i will opt to drop the columns.